In [5]:
from module.shufflenetv2 import ShuffleNetV2
import torch
import torch.nn as nn
class LightClassifier(nn.Module):
    def __init__(self, classes, load_param, debug=False):
        super(LightClassifier, self).__init__()
        
        self.stage_repeats = [4, 8, 4]
        self.stage_out_channels = [-1, 24, 48, 96, 192]
        self.base = ShuffleNetV2(self.stage_repeats, self.stage_out_channels, load_param)

        # Add a global average pooling layer
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Add a fully connected layer for classification
        self.fc = nn.Linear(self.stage_out_channels[-1], classes)
        
        self.debug = debug
        
    def forward(self, x):
        if self.debug :
            print("forward ", x.size())
        _, _, P3 = self.base(x)
        if self.debug :
            print("base output ", P3.size())
        x = self.global_pool(P3)
        if self.debug :
            print("after global pool ", x.size())
        features = x.view(x.size(0), -1)  # Flatten the tensor
        if self.debug :
            print("after tensor flattening ", x.size())
        logits = self.fc(features)
        if self.debug :
            print("final shape ", x.size())
        return features, logits

In [6]:
model = LightClassifier(2, False, True)

model.load_state_dict(torch.load('/home/achintya-trn0175/Downloads/trafficLightClassification/weights/tflt_700epochs/tflt_weight_loss:0.116813_240-epoch.pth'))
dummy_input = torch.rand(1, 3, 192, 192)

Initialize params from:./module/shufflenetv2.pth


In [7]:
torch.onnx.export(model, dummy_input, 'model_weighted_smapler.onnx', export_params=True, opset_version=10, do_constant_folding=True, input_names=['input'], output_names=['output'])

print("Model has been converted to ONNX format.")

forward  torch.Size([1, 3, 192, 192])
base output  torch.Size([1, 192, 6, 6])
after global pool  torch.Size([1, 192, 1, 1])
after tensor flattening  torch.Size([1, 192, 1, 1])
final shape  torch.Size([1, 192, 1, 1])
Model has been converted to ONNX format.


NameError: name 'python' is not defined

In [7]:
from module.shufflenetv2 import ShuffleNetV2
import torch
import torch.nn as nn
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import math
import torch
import argparse
from tqdm import tqdm
from torch import optim
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler, Subset
from utils.tool import *
from utils.datasets import *
from module.loss import DetectorLoss, CombinedLoss
from trafficlight_cls import LightClassifier
from torchvision import datasets, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.datasets import ImageFolder
import numpy as np
from collections import Counter
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import os
import shutil
from module.shufflenetv2 import ShuffleNetV2
import torchvision.transforms.functional as F


print(f"albumentations version: {A.__version__}")
print(f"PIL version: {Image.__version__}")

print(f"numpy version: {np.__version__}")
print(f"argparse version: {argparse.__version__}")



albumentations version: 1.4.10
PIL version: 10.3.0
numpy version: 1.26.4
argparse version: 1.1
